# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
!pip install geoviews

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,shijiang,27.0819,110.7933,7.87,76,40,1.39,CN,1702404895
1,1,navirai,-23.0650,-54.1906,32.56,49,8,2.48,BR,1702404895
2,2,tiksi,71.6872,128.8694,-21.05,100,100,4.81,RU,1702404895
3,3,vila franca do campo,37.7167,-25.4333,15.23,71,38,4.96,PT,1702404896
4,4,manokwari,-0.8667,134.0833,26.46,83,100,2.71,ID,1702404896


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Code to make the map plot and configure it
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    legend = False,
    hover_cols= ["City", "Country", "Lat","Lng","Humidity"],
    tiles = "OSM" 
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df[city_data_df["Humidity"] <= 60]
narrowed_df = narrowed_df[(narrowed_df["Max Temp"] > 20) & (narrowed_df["Max Temp"] < 35)]

# Drop any rows with null values
narrowed_df = narrowed_df.dropna(how='any')

# Display sample data
narrowed_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,navirai,-23.0650,-54.1906,32.56,49,8,2.48,BR,1702404895
19,19,ad dabbah,18.0500,30.9500,20.14,26,51,6.13,SD,1702404898
60,60,aripuana,-9.1667,-60.6333,32.61,51,99,0.63,BR,1702404913
122,122,conde,-7.2597,-34.9075,30.70,55,20,7.20,BR,1702404934
141,141,heroica caborca,30.7167,-112.1500,21.50,12,100,3.36,MX,1702404941
...,...,...,...,...,...,...,...,...,...,...
551,551,kajuru,10.3192,7.6892,20.49,19,3,3.01,NG,1702405059
554,554,de aar,-30.6497,24.0123,20.79,40,5,10.73,ZA,1702405060
561,561,akure,7.2526,5.1931,26.00,41,7,1.33,NG,1702405061
571,571,dwarka,22.2394,68.9678,22.77,56,88,2.54,IN,1702405063


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Reset the index
hotel_df = hotel_df.reset_index(drop=True)

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,navirai,BR,-23.0650,-54.1906,49,
1,ad dabbah,SD,18.0500,30.9500,26,
2,aripuana,BR,-9.1667,-60.6333,51,
3,conde,BR,-7.2597,-34.9075,55,
4,heroica caborca,MX,30.7167,-112.1500,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set search parameters and initialize variables
latitude = 0
longitude = 0
limit = 1
radius = 10000

# Set base URL
base_url = "https://api.geoapify.com/v2/places"
print("Starting restaurant search")

# Iterate through the types_df DataFrame
for index, row in hotel_df.iterrows():
    # Get Lat and Lng for each row in hotel_df
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Set filter and bias for each row in hotel_df
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    # set up a parameters dictionary
    params = {
        "filter":filters,
        "limit":limit,
        "bias":bias,
        "apiKey":geoapify_key    
    }
    # Add the catagory to the parameters list
    params["categories"] = f"accommodation.hotel"
   
    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the hotel from the results and store the details in the DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "hotel not found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting restaurant search
navirai - nearest hotel: Hotel 2 Gaúchos
ad dabbah - nearest hotel: No hotel found
aripuana - nearest hotel: No hotel found
conde - nearest hotel: Paladino
heroica caborca - nearest hotel: Hotel Posada del Desierto
metlili chaamba - nearest hotel: No hotel found
menaka - nearest hotel: No hotel found
reggane - nearest hotel: No hotel found
umluj - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
al hayy - nearest hotel: No hotel found
malkapur - nearest hotel: No hotel found
upata - nearest hotel: Hotel Garibardi
port augusta - nearest hotel: Hotel Commonwealth
bafata - nearest hotel: Apartahotel Maimuna Capé
tazacorte - nearest hotel: App Leyma
arauquita - nearest hotel: No hotel found
al ghayzah - nearest hotel: فندق تاج العرب
kyabe - nearest hotel: No hotel found
caleta olivia - nearest hotel: Grand Hotel Snack
manaure - nearest hotel: hotel palaima
esmeraldas - nearest hotel: No hotel found
santana - nearest hotel: Hotel Teodoro
wajir -

,City,Country,Lat,Lng,Humidity,Hotel Name
0,navirai,BR,-23.0650,-54.1906,49,Hotel 2 Gaúchos
1,ad dabbah,SD,18.0500,30.9500,26,No hotel found
2,aripuana,BR,-9.1667,-60.6333,51,No hotel found
3,conde,BR,-7.2597,-34.9075,55,Paladino
4,heroica caborca,MX,30.7167,-112.1500,12,Hotel Posada del Desierto
...,...,...,...,...,...,...
57,kajuru,NG,10.3192,7.6892,19,No hotel found
58,de aar,ZA,-30.6497,24.0123,40,No hotel found
59,akure,NG,7.2526,5.1931,41,Hotel Continental
60,dwarka,IN,22.2394,68.9678,56,The Dwarika Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display
# Bonus step: Drop results where there is no hotel within 10000m
cleaned_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == "No hotel found"].index)

# Create and configure the map plot
map_plot_2 = cleaned_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    frame_width = 800,
    frame_height = 600,
    color = "City",
    hover_cols= ["City", "Country", "Humidity", "Hotel Name"],
    legend = False,
    tiles = "OSM" 
)

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)